This is the code for the Machine Learning Model for the Vancouver Housing Affordability project.
To do a basic test you can simply run all the cells in order. For more detailed testing instructions refer
to the README file.

Please note that this code requires two files "ML-Model-Cleaned.csv" and "static_params.txt" to be saved in the same folder, and won't work otherwise.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import os

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#Loading the initial data from the file.
filename = "ML-Model-Cleaned.csv"
data = pd.read_csv(filename).dropna()

In [15]:
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#CHANGABLE MODEL PARAMETERS
#These are the only values in the notebook that should be
#changed. Each time you change one of these you need to
#re-run all the cells below in order (including re-training
#the ML model) for the change to take effect.
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
#LOOKBACK_YEARS Parameter determines how many previous
#of property values get included in each feature vector.
#For example, if LOOKBACK_YEARS = 5 then the feature vector
#for predicting 2025 prices will include property valeus
#from 2020-2024. Should never be higher than 19. See
#next parameter for additional restrictions.
#Can be set to 0 to predict without using property values.
#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
LOOKBACK_YEARS = 0

#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
#TRAINING_YEARS parameter lists which years of data 
#data the model will try to predict as part of training.
#For any given year, the model will try to predict every
#property in that year (except those held out for testing).
#Adding more years means more training data, but using
#only 2024 is sufficient to get decent accuracy.
#Only years in the range 2006-2024 should ever be used, but
#if using the early end of the range, you need to also make
#sure that you're not looking back beyond it. For example, if
#LOOKBACK_YEARS = 5, then you can't train on any year earlier
#than 2010 (since the 2009 df would try to incorporate 2005)
#property values, which we don't have.
#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
TRAINING_YEARS = [2013, 2017, 2024]

In [16]:
#GLOBAL CONSTANTS AND FILE INPUTS
#This cell defines global constants plus a line that reads a file to
#determine which 'static columns' (columns that don't depend on year)
#to use. Needs to have correctly-named file in the same folder to run
#without errors.


LATEST_DATA_YEAR = 2024
FIRST_DATA_YEAR = 2006
IMPROVEMENT_PREFIX = "CURRENT_IMPROVEMENT_VALUE_"
IMPROVEMENT_SUFFIX = "_YEAR_AGO_IMPROVEMENT_VALUE"
LAND_PREFIX = "CURRENT_LAND_VALUE_"
LAND_SUFFIX = "_YEAR_AGO_LAND_VALUE"
OCCUPANCY_PREFIX = "Count_Per_Year_"
Y_COLS = ["IMPROVEMENT_VALUE", "LAND_VALUE"]

pd.options.mode.copy_on_write = True

f = open("static_params.txt", "r") 
f_params = f.read() 
stat_cols = f_params.replace('\n', ',').split(',')
f.close()
num_cols = len(stat_cols) - 1
stat_cols = stat_cols[:num_cols] #remove empty string in last position

In [17]:
#DATA LOADING AND SHAPING FUNCTIONS
#Function in this cell are used to filter and format the raw data
#from the CSV file into the correct format and shape to be used
#by the ML model.

#Input a year (among the years we have data for) and selects all relevant
#data for that year into a dataframe, altering the column names to a
#consistent standard that doesn't depend on the year.
#If you pass in a year that's too near the start (as determined by global)
#constant LOOKBACK_YEARS, you'll get an error
def get_year_data(pred_year):
    year_cols = [] #generate column names for all the year-dependent stuff
    new_col_names = {}
    
    #We need to change our column names from having absolute years (e.g. 2017)
    #to relative years (7 years ago) so we make a dictionary relating the two
    for year in range(pred_year - LOOKBACK_YEARS, pred_year):
        imp, land = IMPROVEMENT_PREFIX + str(year), LAND_PREFIX + str(year)
        year_cols += [imp, land]
        new_col_names.update({imp:str(pred_year - year) + IMPROVEMENT_SUFFIX})
        new_col_names.update({land:str(pred_year - year) + LAND_SUFFIX})
    
    #only save one year of occupancy data
    occup = OCCUPANCY_PREFIX + str(pred_year - 1)
    year_cols.append(occup)
    new_col_names.update({occup:"OCCUPANCY"})
    
    #Most recent columns of property data get special names, since they are 
    #the "current" values and thus our training labels/prediction targets
    X_cols = stat_cols + year_cols
    yimp, yland = IMPROVEMENT_PREFIX + str(pred_year), LAND_PREFIX + str(pred_year)
    y_cols_temp = [yimp, yland]
    
    #Copy the columns we want from the original df and then rename them
    prepped_df = data[X_cols + y_cols_temp]
    prepped_df.rename(columns = new_col_names, inplace=True)
    prepped_df.rename(columns={yimp:Y_COLS[0], yland:Y_COLS[1]},inplace=True)
    prepped_df.insert(0,'YEAR',pred_year)
    
    return prepped_df

#Splits a prepped dataframe into training and testing sets
#and into X and y components
def split_data(df):
    train, test = train_test_split(full_df,test_size=0.2)
    y_train = train[Y_COLS]
    y_test = test[Y_COLS]
    X_train = train.drop(Y_COLS, axis=1)
    X_test = test.drop(Y_COLS, axis=1)
    
    return X_train, y_train, X_test, y_test

#Takes in a list of years, calls get_year_data for each
#of those years and combines them
def combine_years(years):
    empty = True
    for year in years:
        if(empty):
            df = get_year_data(year)
            empty = False
        else:
            more_data = get_year_data(year)
            df = pd.concat([df,more_data],axis=0)
            
    return df
        



In [18]:
#TRAINING AND TESTING ML MODEL
#This cell gathers the data to feed into the model
#Calls the function to split it into train and test
#sets and then trains and scores the model

full_df = combine_years(TRAINING_YEARS)

X_train, y_train, X_test, y_test = split_data(full_df)

rf = RandomForestRegressor()
rf.fit(X_train, y_train)

score = rf.score(X_test, y_test)

print(score)

0.702466258940381


In [19]:
#FEATURE IMPORTANCE
#Displays feature importances in a readable form
print("Feature Importances for RandomForest Model:")
for i in range(len(X_test.columns)):
    print(X_test.columns[i], ": ", 100*rf.feature_importances_[i], "%")

Feature Importances for RandomForest Model:
YEAR :  9.167693362389665 %
YEAR_BUILT :  3.3330998698232777 %
BIG_IMPROVEMENT_YEAR :  4.033745614464224 %
LATITUDE :  7.520921399886735 %
LONGITUDE :  9.285415735374668 %
MIN_DISTANCE_LIBRARY_METERS :  3.5549344242262624 %
COMMUNITY_CENTRE_DISTANCE_METERS :  2.938592770106727 %
CLOSEST_DOG_PARK_METERS :  4.8479505374431415 %
CLOSEST_HOMELESS_SHELTER :  3.162412281969572 %
CLOSEST_PUBLIC_ART :  7.520654722122804 %
CLOSEST_RAPID_TRANSIST :  4.906247847621237 %
CLOSEST_BUS_STOPS :  5.4051228743948 %
SCHOOL_DISTANCE :  3.9414346099109725 %
PARK_DISTANCE :  4.0089614097981645 %
PARK_AREA_HECTARES :  1.9891439557957036 %
CULTURAL_SPACE_YEAR :  0.9148303240988701 %
CULTURAL_SPACE_DISTANCE :  3.624197249054015 %
OCCUPANCY :  19.84464101151916 %


In [8]:
#FUNCTIONS FOR PREDICTING FUTURE YEARS
#The two function in this cell are used
#to allow the model to make predictions on
#future years

#Takes in an X dataframe and a y numpy array and updates
#the dataframe to include the new values, pushing out the
#oldest year of data to make room
#Seems to have some sort of error that causes some small
#number of props not to be correctly updated and getting NAN 
#values instead. We didn't have time to chase down the
#error, so it's patched by simply dropping the affected 
#rows in the predict_future_year rows function
def update_prop_values(old_df, new_prop_data):
    updata = old_df
    
    for i in range(LOOKBACK_YEARS - 1):
        updata[str(LOOKBACK_YEARS-i) + IMPROVEMENT_SUFFIX]\
        = updata[str(LOOKBACK_YEARS-i-1) + IMPROVEMENT_SUFFIX]
        
        updata[str(LOOKBACK_YEARS-i) + LAND_SUFFIX]\
        = updata[str(LOOKBACK_YEARS-i-1) + LAND_SUFFIX]
        
    pred_df = pd.DataFrame(data=new_prop_data, columns=Y_COLS)
    updata['1'+IMPROVEMENT_SUFFIX] = pred_df[Y_COLS[0]]
    updata['1'+LAND_SUFFIX] = pred_df[Y_COLS[1]]
    
    return updata

#Makes predictions for some future year. If the year is more
#than one year in advance, it will recursively predict each
#year between that year and 2024. Returns both the predictions
#and the updata dataframe (the latter being necessary so it)
#can feed it forward to predict further-future years
def predict_future_year(model, year):
    
    #If we're predicting just one year in the future we can
    #just use values from our existing dataframe
    if(year == LATEST_DATA_YEAR + 1):
        last_year = get_year_data(year-1)
        prev_data = last_year.drop(Y_COLS, axis=1)
        prev_predictions = last_year[Y_COLS]
    
    #If we're predicting farther in the future, we need to fill in all
    #years between our latest data and the year we're predicting 
    elif(year > LATEST_DATA_YEAR + 1):
        prev_data,prev_predictions  = predict_future_year(model,year-1)
    
    #Update property data (if necessary), advance the year and
    #Make next year's predictions
    if(LOOKBACK_YEARS > 0):
        prev_data = update_prop_values(prev_data,prev_predictions)
    prev_data['YEAR'] = prev_data['YEAR'].apply(lambda x: x+1)    
    predictions = model.predict(prev_data.dropna())
    
    return prev_data, predictions


In [9]:
#TESTING THE MODEL AND MAKING PREDICTIONS
#This is the cell where the model can actually be
#used to predict future property values. You can
#change future_prediction_year to any year after
#2024 to predict future property values

#To inspect the results in Jupyter Notebook,
#uncomment one of the three lines at the end to 
#see either just the predictions, just the updated
#dataframe or both together

future_prediction_year = 2027

updata, preds = predict_future_year(rf, future_prediction_year)
pred_df = pd.DataFrame(data=preds, columns=Y_COLS)

#pred_df
#updata.dropna()
#pd.concat([updata, pred_df], axis=1).dropna()